In [21]:
import mdtraj as md
import nglview as nv

molecule="2jof"
molecule_lower = molecule.lower()
molecule_upper = molecule.upper()

# Visualization

In [19]:
pdb_path = f"../data/{molecule_upper}/folded.pdb"
gro_path = f"../data/{molecule_upper}/folded.gro"
traj = md.load(gro_path)
view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view.display()

NGLWidget()

In [20]:
ca_atoms = traj.topology.select("name CA")
print(ca_atoms)

[  4  16  26  47  57  74  98 117 127 139 146 161 167 178 189 196 228 242
 256 262]


In [15]:
for atom in traj.topology.atoms:
    print(f"Index: {atom.index:4d} | Name: {atom.name:>4s} | Element: {atom.element} | Residue: {atom.residue}")

Index:    0 | Name:    N | Element: nitrogen | Residue: ASP1
Index:    1 | Name:    H | Element: hydrogen | Residue: ASP1
Index:    2 | Name:   H2 | Element: hydrogen | Residue: ASP1
Index:    3 | Name:   H3 | Element: hydrogen | Residue: ASP1
Index:    4 | Name:   CA | Element: carbon | Residue: ASP1
Index:    5 | Name:   HA | Element: hydrogen | Residue: ASP1
Index:    6 | Name:   CB | Element: carbon | Residue: ASP1
Index:    7 | Name:  HB3 | Element: hydrogen | Residue: ASP1
Index:    8 | Name:  HB2 | Element: hydrogen | Residue: ASP1
Index:    9 | Name:   CG | Element: carbon | Residue: ASP1
Index:   10 | Name:  OD1 | Element: oxygen | Residue: ASP1
Index:   11 | Name:  OD2 | Element: oxygen | Residue: ASP1
Index:   12 | Name:    C | Element: carbon | Residue: ASP1
Index:   13 | Name:    O | Element: oxygen | Residue: ASP1
Index:   14 | Name:    N | Element: nitrogen | Residue: ALA2
Index:   15 | Name:    H | Element: hydrogen | Residue: ALA2
Index:   16 | Name:   CA | Element: ca

In [ ]:
# traj = md.load(f"../data/{molecule_upper}/1FME.pdb")
# traj.save_gro(f"../data/{molecule_upper}/folded.gro")

# GROMACS CMDs

In [22]:
print("\nCreate .gro file")
# print(f"gmx pdb2gmx -f {molecule_upper}/folded.pdb -o {molecule_upper}/folded.gro -p {molecule_upper}/topol.top -ignh")
print(f"gmx pdb2gmx -f {molecule_upper}/folded.pdb -o {molecule_upper}/folded.gro -p {molecule_upper}/topol.top -ff charmm27 -water tip3p --ignh")

print("\nCreate .tpr file")
print(f"gmx grompp -f {molecule_lower}.mdp -c {molecule_upper}/folded.gro -p {molecule_upper}/topol.top -o {molecule_upper}/nvt_0.tpr -maxwarn 2")


Create .gro file
gmx pdb2gmx -f 2JOF/folded.pdb -o 2JOF/folded.gro -p 2JOF/topol.top -ff charmm27 -water tip3p --ignh

Create .tpr file
gmx grompp -f 2jof.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2JOF/nvt_0.tpr -maxwarn 2


In [ ]:
gmx editconf -f protein.gro -o boxed.gro -c

gmx grompp -f test.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2jof.tpr

gmx pdb2gmx -f 2JOF/folded.pdb -o 2JOF/folded.gro -p 2JOF/topol.top -ff amber99sb-ildn -water tip3p

gmx grompp -f 2JOF/2jof.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2jof.tpr

gmx grompp -f test.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2jof.tpr

In [ ]:
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.c.tdb

/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb

residuetypes.dat 

In [ ]:
1.79071   2.10721   1.56292

8: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

1: TIP3P   TIP 3-point, recommended